<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/chat_bot_gpu_worker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version

# install

In [ ]:
!pip install "achatbot[remote_queue_chat_bot_be_worker]"="==0.0.5.27"


In [ ]:
!pip show scipy

# download models

In [ ]:
!mkdir -p ~/.achatbot/{log,config,models,records,videos}

In [ ]:
!git lfs install


In [ ]:
# vad pyannote segmentation ckpt
#!huggingface-cli download pyannote/segmentation-3.0  --local-dir ~/.achatbot/models/pyannote/segmentation-3.0 --local-dir-use-symlinks False

In [ ]:
# asr openai whisper ckpt
!wget https://openaipublic.azureedge.net/main/whisper/models/ed3a0b6b1c0edf879ad9b11b1af5a0e6ab5db9205f891f668f8b0e6c6326e34e/base.pt -O ~/.achatbot/models/base.pt

# asr hf openai whisper ckpt for transformers pipeline to load
!huggingface-cli download openai/whisper-base  --local-dir ~/.achatbot/models/openai/whisper-base --local-dir-use-symlinks False

# asr hf faster whisper (CTranslate2)
!huggingface-cli download Systran/faster-whisper-base  --local-dir ~/.achatbot/models/Systran/faster-whisper-base --local-dir-use-symlinks False

# asr SenseVoice ckpt
!huggingface-cli download FunAudioLLM/SenseVoiceSmall  --local-dir ~/.achatbot/models/FunAudioLLM/SenseVoiceSmall --local-dir-use-symlinks False

In [ ]:
# llm llamacpp Qwen2-Instruct
!huggingface-cli download Qwen/Qwen2-1.5B-Instruct-GGUF qwen2-1_5b-instruct-q8_0.gguf  --local-dir ~/.achatbot/models --local-dir-use-symlinks False

# llm llamacpp Qwen1.5-chat
!huggingface-cli download Qwen/Qwen1.5-7B-Chat-GGUF qwen1_5-7b-chat-q8_0.gguf  --local-dir ~/.achatbot/models --local-dir-use-symlinks False

# llm llamacpp phi-3-mini-4k-instruct
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-gguf Phi-3-mini-4k-instruct-q4.gguf --local-dir ~/.achatbot/models --local-dir-use-symlinks False


In [ ]:
# tts chatTTS
!huggingface-cli download 2Noise/ChatTTS  --local-dir ~/.achatbot/models/2Noise/ChatTTS --local-dir-use-symlinks False

# tts coquiTTS
!huggingface-cli download coqui/XTTS-v2  --local-dir ~/.achatbot/models/coqui/XTTS-v2 --local-dir-use-symlinks False

# tts cosy voice
!git lfs install
!git clone https://www.modelscope.cn/iic/CosyVoice-300M.git ~/.achatbot/models/CosyVoice-300M
!git clone https://www.modelscope.cn/iic/CosyVoice-300M-SFT.git ~/.achatbot/models/CosyVoice-300M-SFT
!git clone https://www.modelscope.cn/iic/CosyVoice-300M-Instruct.git ~/.achatbot/models/CosyVoice-300M-Instruct
#git clone https://www.modelscope.cn/iic/CosyVoice-ttsfrd.git ~/.achatbot/models/CosyVoice-ttsfrd


# run

   u can login [redislabs](https://app.redislabs.com/#/) create 30M free databases; set `REDIS_HOST`,`REDIS_PORT` and `REDIS_PASSWORD` to run

In [ ]:
from google.colab import userdata
redis_pwd = userdata.get('REDIS_PASSWORD')
#print(redis_pwd)

In [ ]:
!ACHATBOT_PKG=1 RUN_OP=be \
  TQDM_DISABLE=True \
  REDIS_PASSWORD=$redis_pwd \
  REDIS_HOST=redis-14241.c256.us-east-1-2.ec2.redns.redis-cloud.com \
  REDIS_PORT=14241 \
  ASR_TAG=sense_voice_asr \
  ASR_LANG=zn \
  ASR_MODEL_NAME_OR_PATH=~/.achatbot/models/FunAudioLLM/SenseVoiceSmall \
  N_GPU_LAYERS=33 FLASH_ATTN=1 \
  LLM_MODEL_NAME=qwen \
  LLM_MODEL_PATH=~/.achatbot/models/qwen1_5-7b-chat-q8_0.gguf \
  TTS_TAG=tts_cosy_voice \
  python -m achatbot.cmd.remote-queue-chat.generate_audio2audio > ~/.achatbot/log/be_std_out.log
